# Generalizing the Pipeline

<div class="alert alert-block alert-info">
<b>Warning:</b>
This notebook depends on the Parquet files generated by the notebook <b>01 Preparing the Data</b>. Make sure to run all cells in that notebook before executing this one.
    
In particular, the files required are:
<ul>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet</tt></li>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet</tt></li>
</ul>
</div>

In [1]:
# This cell does not need to be run on the Bodo.ai platform and should be skipped
import ipyparallel as ipp, psutil
n_proc = min(psutil.cpu_count(logical=False), 8)
rc = ipp.Cluster(engines='mpi', n=n_proc).start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 4/4 [00:05<00:00,  1.48s/engine]


<div class="alert alert-block alert-info">
<b>Note:</b>

Having launched IPyParallel, the `px` magics ensure that code is executed on all engines.
    
1. The `%px` line magic executes a single line of code an all engines.
2. The `%%px` cell magic—placed as the first line of a cell—ensures all lines in a given cell are executed on all engines.
3. Lines executed outside of the context of a `px` line or cell magic execute in the namespace of the default Jupyter kernel.

Consult the [IPyParallel documentation](https://ipyparallel.readthedocs.io/en/latest/) for more information about setting up a local cluster. [Bodo's documentation](https://docs.bodo.ai/2022.2/bodo_parallelism/bodo_parallelism_basics/) provides more information about the parallel execution model.
</div>

In [2]:
%%px

import numpy as np, pandas as pd
import time, bodo

%px: 100%|██████████| 4/4 [00:01<00:00,  3.17tasks/s]


## A First Attempt: specifying a toggle

There's one more important thing we've not yet considered: how to specify different input files to use as input. Keep in mind that the Bodo JIT compiler must statically type objects for compilation. That is, `bodo.jit` will do everything it can to type-infer dtypes in the process, if there is a potential conflict the compile logic will exit rather than attempt to force a compatibility. 

We can see this in the next example.  We'll set up `load_parking_tickets_toggle` to use a boolean input parameter as a toggle value to use in an `if/else` branch within the function. Perhaps an earlier piece of our data pipeline supplies this toggle value and we want this function to vary its behavior depending on that input.

In [3]:
%%px

@bodo.jit()
def load_parking_tickets_toggle(toggle):
    """
    Load data from file and aggregate by day, violation type, and police precinct.
    """

    start = time.time()
    if toggle:
        year_2016_df = pd.read_parquet('ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet')
        year_2016_df = year_2016_df.groupby(['Issue Date','Violation County','Violation Precinct','Violation Code'], as_index=False)['Summons Number'].count()
        many_year_df = year_2016_df
    else:
        year_2017_df = pd.read_parquet('ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet')
        year_2017_df = year_2017_df.groupby(['Issue Date','Violation County','Violation Precinct','Violation Code'], as_index=False)['Summons Number'].count()
        many_year_df = year_2017_df

    end = time.time()
    timing_str = f"\n{'Reading Time:':<42}{end - start:8.3f} sec"
    return many_year_df, timing_str

When we attempt to invoke `load_parking_tickets_toggle`, an exception is raised. We have deleted the lengthy output.

<div class="alert alert-block alert-info">
<b>Warning:</b>
Executing the next cell is expected to yield errors (on all engines, as is usual in `ipyparallel`). You will see a lengthy stack trace if you attempt to execute the notebook from top to bottom (e.g., using the <tt>Run All Cells</tt> option from the <tt>Run</tt> menu).
</div>

In [ ]:
%%px
# This cell is expected to fail when executed.

load_parking_tickets_toggle(False)

The important part of the lengthy stack trace produced looks like this:

<div class="alert alert-block alert-info">
<tt>TypingError: Cannot unify dataframe((array(datetime64[ns], 1d, C), StringArrayType(), array(int64, 1d, C), array(int64, 1d, C), array(int64, 1d, C)), RangeIndexType(none), ('Issue Date', 'Violation County', 'Violation Precinct', 'Violation Code', 'Summons Number'), 1D_Block_Var, False) and dataframe((array(datetime64[ns], 1d, C), StringArrayType(), array(float64, 1d, C), array(int64, 1d, C), array(int64, 1d, C)), RangeIndexType(none), ('Issue Date', 'Violation County', 'Violation Precinct', 'Violation Code', 'Summons Number'), 1D_Block_Var, False) for 'many_year_df.2'</tt>
</tt>
</div>

The problem is that the `'Violation Precinct'` column is inferred as dtype `int64` for the `2016` dataset, and as dtype `float64` for the `2017` dataset (likely due to missing entries cast as `NaN`).  As we assign either `year_2016_df` or `year_2017_df` to `many_year_df`, the compiler output is not *type-stable*, i.e., it does not have a single clear schema for `many_year_df`.

## A Second Approach: specifying a file path

We can, if absolutely neccessary, explicitly specify dtypes for the specific files; the process is described [here](https://docs.bodo.ai/2022.3/file_io/#non-constant-filepaths). A far easier path is to take the ambiguity out in the first place and let the compiler handle the request on a case-by-case basis. This is the approach we settle on here in `load_parking_tickets_file`.

In [5]:
%%px
@bodo.jit
def load_parking_tickets_file(file):
    """
    Load data from specified file and aggregate by day, violation type, and police precinct.
    """
    start = time.time()
    year_df = pd.read_parquet(file)
    groupby_cols = ['Issue Date','Violation County','Violation Precinct','Violation Code']
    year_df = year_df.groupby(groupby_cols, as_index=False)['Summons Number'].count()

    end = time.time()
    timing_str = f"\n{'Reading Time:':<42}{end - start:8.3f} sec"
    return year_df, timing_str

This executes smoothly on the data from 2016.

In [6]:
%%px
# Try executing again with the 2016 file path as input
DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet'
df, msg = load_parking_tickets_file(DATA_SRC)
print(df.shape)
if bodo.get_rank()==0:
    print(msg)

%px:   0%|          | 0/4 [00:07<?, ?tasks/s]

[stdout:3] (157905, 5)


[stdout:1] (158634, 5)


[stdout:0] (158117, 5)

Reading Time:                                1.642 sec


[stdout:2] (158591, 5)


%px: 100%|██████████| 4/4 [00:07<00:00,  1.86s/tasks]


We can try again with another input file, say the data from 2017, as input.

In [7]:
%%px
# Try executing again with the 2017 file path as input
DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet'
df, msg = load_parking_tickets_file(DATA_SRC)
print(df.shape)
if bodo.get_rank()==0:
    print(msg)

%px:   0%|          | 0/4 [00:02<?, ?tasks/s]

[stdout:2] (155818, 5)


[stdout:1] (155662, 5)


[stdout:3] (156501, 5)


[stdout:0] (156407, 5)

Reading Time:                                2.085 sec


%px: 100%|██████████| 4/4 [00:02<00:00,  1.60tasks/s]


We could even tweak the function to have multiple arguments passed to the function if we need to concatenate multiple files together. Bodo also supports lists of Parquet files passed in as arguments—or requesting an entire folder be read in from CSV or Parquet formats.

Or perhaps, in keeping with the logic of the toggled function earlier, we can execute the toggling logic outside of a `bodo.jit`-compiled function and let the compiler handle specific cases without ambiguity.

In [8]:
%%px

## notice no jit decorator here
def run_load_by_toggle(toggle):
    if toggle:
        DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet'
    else:
        DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet'
    return load_parking_tickets_file(DATA_SRC)

In [9]:
%%px

result, msg = run_load_by_toggle(True)
if bodo.get_rank()==0:
    display(result.head())
    print(msg)

[output:0]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number
0,2015-07-09,K,94.0,70,11
1,2015-07-09,K,94.0,21,134
2,2015-07-09,K,84.0,21,53
3,2015-07-09,K,84.0,37,76
4,2015-07-09,K,90.0,21,134


[stdout:0] 
Reading Time:                                1.225 sec


We now have most of what we need to build a flexible pipeline. Should we need it, Bodo also has the useful built-in `bodo.typeof` that can be used to determine what types are being inferred by the compiler, so we don't get into unexpected situations.

In [10]:
%%px

my_temp_df = load_parking_tickets_file('ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet')
if bodo.get_rank()==0:
    print(bodo.typeof(my_temp_df))

[stdout:0] Tuple(dataframe((array(datetime64[ns], 1d, C), StringArrayType(), array(float64, 1d, C), array(int64, 1d, C), array(int64, 1d, C)), RangeIndexType(none), ('Issue Date', 'Violation County', 'Violation Precinct', 'Violation Code', 'Summons Number'), 1D_Block_Var, False), unicode_type)


Remember to cleanly shut down the `ipyparallel` session:

In [11]:
# To stop the cluster run the following command.
rc.cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 1087, 'identifier': 'ipcontroller-1651457258-no50-1060'}
Stopping engine(s): 1651457259
engine set stopped 1651457259: {'exit_code': 0, 'pid': 1125, 'identifier': 'ipengine-1651457258-no50-1651457259-1060'}
